In [6]:
import time
import mlrose_hiive as mlrh
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [7, 7]

random_state = 923
np.random.seed(923)

# Knapsack Problem

In [7]:
# Problem definition for comparison
number_of_items_types = 40
max_weight_per_item = 25
max_value_per_item = 15

weights = 1 + np.random.randint(max_weight_per_item, size=number_of_items_types)
values = 1 + np.random.randint(max_value_per_item, size=number_of_items_types)
max_weight_pct = 0.35
max_item_count = 2

global eval_count
eval_count = 0

base_fit_func = mlrh.Knapsack(weights, values, max_weight_pct=max_weight_pct).evaluate

def knapsack_fit(state):
    global eval_count
    eval_count += 1

    return base_fit_func(state)


# Initialize custom fitness function object
fitness = mlrh.CustomFitness(knapsack_fit)

problem = mlrh.KnapsackOpt(length=number_of_items_types,
                        max_val=max_item_count,
                        max_weight_pct=max_weight_pct,
                        fitness_fn=fitness,
                        maximize=True)

experiment_name = "knapsack"
output_directory = "knapsack"

## MIMIC

In [3]:
mim = mlrh.MIMICRunner(problem=problem,
                       experiment_name=experiment_name,
                       output_directory=output_directory,
                       seed=random_state,
                       population_sizes=[50, 100, 200, 500, 1000, 2000],
                       keep_percent_list=[0.1, 0.25, 0.5, 0.7, 0.9],
                       iteration_list=[50],
                       use_fast_mimic=True)
mim_stats, mim_curve = mim.run()
show=10
columns = ['Time', 'Fitness', 'Population Size', 'Keep Percent']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./knapsack/knapsack/mimic__knapsack__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Population Size'], ascending=(False,True)).head(show)

,Time,Fitness,Population Size,Keep Percent
41,2.841594,196.0,1000,0.10
43,2.913932,196.0,1000,0.25
45,2.851498,196.0,1000,0.50
47,2.875752,196.0,1000,0.70
49,2.848349,196.0,1000,0.90
51,5.332814,189.0,2000,0.10
53,5.349505,189.0,2000,0.25
55,5.354411,189.0,2000,0.50
57,5.422686,189.0,2000,0.70
59,5.410722,189.0,2000,0.90


## Genetic Algorithm

In [4]:
ga = mlrh.GARunner(problem=problem,
                   experiment_name=experiment_name,
                   output_directory=output_directory,
                   seed=random_state,
                   max_attempts=20,
                   population_sizes=[50, 100, 200, 500, 1000, 2000],
                   mutation_rates=[0.1, 0.25, 0.5, 0.7, 0.9],
                   iteration_list=[50])
ga_stats, ga_curve = ga.run()
show=10
columns = ['Time', 'Fitness', 'Population Size', 'Mutation Rate']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./knapsack/knapsack/ga__knapsack__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Mutation Rate'], ascending=(False,True)).head(show)

,Time,Fitness,Population Size,Mutation Rate
21,0.587025,229.0,200,0.10
31,1.323961,229.0,500,0.10
41,2.838979,229.0,1000,0.10
51,5.591841,229.0,2000,0.10
23,0.592602,229.0,200,0.25
33,1.391606,229.0,500,0.25
43,2.705820,229.0,1000,0.25
53,5.602342,229.0,2000,0.25
25,0.611333,229.0,200,0.50
35,1.380318,229.0,500,0.50


## Simulated Annealing

In [8]:
sa = mlrh.SARunner(problem=problem,
                   experiment_name=experiment_name,
                   output_directory=output_directory,
                   seed=random_state,
                   max_attempts=200,
                   iteration_list=[1000],
                   temperature_list=[0.05, 0.1, 0.5, 1, 10, 30],
                   decay_list=[mlrh.GeomDecay, mlrh.ExpDecay, mlrh.ArithDecay])
sa_stats, sa_curve = sa.run()

show=10
columns = ['Time', 'Fitness', 'Temperature', 'schedule_type']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./knapsack/knapsack/sa__knapsack__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Temperature'], ascending=(False,True)).head(show)

,Time,Fitness,Temperature,schedule_type
27,0.089260,191.0,10.00,exponential
31,0.115243,188.0,30.00,geometric
33,0.171609,186.0,30.00,exponential
25,0.074457,164.0,10.00,geometric
17,0.117148,153.0,0.50,arithmetic
23,0.118902,153.0,1.00,arithmetic
29,0.213177,151.0,10.00,arithmetic
1,0.048492,140.0,0.05,geometric
3,0.041140,140.0,0.05,exponential
5,0.046986,140.0,0.05,arithmetic


# Randomized Hill Climbing

In [13]:
rhc = mlrh.RHCRunner(problem=problem,
                    experiment_name=experiment_name,
                    output_directory=output_directory,
                    seed=random_state,
                    max_attempts=200,
                    iteration_list=[1000],
                    restart_list=[5,10])
rhc_stats, rhc_curve = rhc.run()
show=10
columns = ['Time', 'Fitness', 'Restarts', 'current_restart']
pd.set_option('display.max_rows',show)
df=pd.read_csv("./knapsack/knapsack/rhc__knapsack__run_stats_df.csv")
df[columns].sort_values(by=['Fitness','Restarts'], ascending=(False,True)).head(show)

,Time,Fitness,Restarts,current_restart
7,0.243339,149.0,5,3
19,0.290357,149.0,10,3
0,0.000136,0.0,5,0
1,0.033326,0.0,5,0
2,0.051396,0.0,5,1
3,0.086184,0.0,5,1
4,0.103180,0.0,5,2
5,0.149171,0.0,5,2
6,0.167263,0.0,5,3
8,0.262518,0.0,5,4
